In [14]:
import numpy as np
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.models import load_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

In [15]:
altura, longitud = 30, 30 #Tamaño de las imagenes
modelo = './modelo/modelo.h5'
pesos = './modelo/pesos.h5'
cnn = load_model(modelo)
cnn.load_weights(pesos)
datos_entrenamiento = './Señales/Entrenamiento'
datos_validacion = './Señales/Prueba'
batch_size = 32

In [16]:
entrenamiento_datagen = ImageDataGenerator( #Set de datos de entrenamiento
    rescale=1./255, #Cada uno de los pixeles tiene un rango de 0 a 255, en este caso todos los valores de pixeles tendran valores de 0 a 1
    shear_range=0.3, #Inclina la imagen para que el algoritmo aprenda a reconocer imagenes en todos los sentidos
    zoom_range=0.3, #Hace zoom para que aprenda a que no siempre aparecen "las señales completos"
    horizontal_flip = True #Toma una imagen y la invierte para que la red neuronal aprenda direccionalidad
)

validacion_datagen = ImageDataGenerator( #Set de datos de validacion
    rescale=1. / 255, # Cada uno de los pixeles tiene un rango de 0 a 255, en este caso todos los valores de pixeles tendran valores de 0 a 1
                      #Solo se las escala porque para validarlas no queremos hacerle zoom, girarla ni nada de eso
)

imagen_entrenamiento = entrenamiento_datagen.flow_from_directory( #Entra al directorio data_entrenamiento, abre todas las carpetas, las procesa a altuna y longitud especificada
    #las procesa en un batch_size de 32, y class_mode = 'categorical' es porque es una clasificacion categorica
    datos_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical'
)

imagen_validacion = validacion_datagen.flow_from_directory(
    datos_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4284 images belonging to 60 classes.
Found 2411 images belonging to 60 classes.


In [17]:
def predecir (file):
    x = load_img(file, target_size = (longitud, altura)) #Le cargo la imagen y le asigno una altura y una longitud
    x = img_to_array(x) #La convierto en un arreglo
    x = np.expand_dims(x, axis = 0) #En el eje 0, le agregamos una nueva dimensión para poder procesar la información sin problema
    arreglo = cnn.predict(x) #Llamamos a la red para que haga una predicción. Devuelve un arreglo de dos dimensiones  

    return np.argmax(arreglo[0])
    

In [18]:
acierto = 0
fallo = 0
cantidad_imagenes = 2411
for i in range(cantidad_imagenes):
    x = int(predecir(imagen_validacion.filepaths[i]))
    y = int(imagen_validacion.filenames[i][3] + imagen_validacion.filenames[i][4])
    if x == y:
        acierto = acierto + 1
    else: 
        fallo = fallo + 1
print ("Acierto: " + str(acierto))
print ("Fallo: " + str(fallo))

print("Porcentaje de acierto: " + str(100-(fallo*100)/cantidad_imagenes))
print("Porcentaje de fallo: " + str((fallo*100)/cantidad_imagenes)) 

Acierto: 2277
Fallo: 134
Porcentaje de acierto: 94.4421401907922
Porcentaje de fallo: 5.557859809207797
